# Анализ сервиса для чтения книг по подписке(SQL)</div>


</b> В нашем распоряжении база данных сервиса для чтения книг по подписке, содержащая информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Необходим проанализировать представленные данные для определения ценностного предложения для нового приложения для любителей чтения. Данные мы получим из таблиц `books`, `authors`, `publishers`, `ratings` и `reviews`.</div>


 #### Задания:</div>


 - Посчитать, сколько книг вышло после 1 января 2000 года;
 - Для каждой книги посчитать количество обзоров и среднюю оценку;
 - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
 - Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
 - Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine #загрузим необходимые библиотеки

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                          db_config['pwd'],
                                                          db_config['host'],
                                                          db_config['port'],
                                                          db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) #подготовим подключение к базе

Для начала посмотрим на представленные таблицы с данными: выведем первые строки представленных таблиц, определим сколько строк составляют таблицу, присутствуют ли дубликаты и какие типы данных представлены.

In [3]:
query = '''
           SELECT *
           FROM books
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
           SELECT COUNT(*)
           FROM books
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,count
0,1000


In [5]:
query_d = '''
           SELECT * 
           FROM books 
           WHERE ctid NOT IN (SELECT max(ctid) FROM books GROUP BY book_id)
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id


In [6]:
query_d = '''
           SELECT column_name, column_default, data_type 
           FROM INFORMATION_SCHEMA.COLUMNS 
           WHERE table_name = 'books';
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,column_name,column_default,data_type
0,book_id,None,integer
1,author_id,None,integer
2,title,None,text
3,num_pages,None,integer
4,publication_date,None,date


Таблица books содержит информацию о книгах, в частности индентификаторы книг и авторов, названия книг, количество страниц, дату публикации и идентификатор издателя. Количество строк - 1000. Данные представлены в корректных форматах, дубликатов не имеется.

In [7]:
query = '''
           SELECT *
           FROM authors
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = '''
           SELECT COUNT(*)
           FROM authors
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,count
0,636


In [9]:
query_d = '''
           SELECT * 
           FROM authors 
           WHERE ctid NOT IN (SELECT max(ctid) FROM authors GROUP BY author_id)
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,author_id,author


In [10]:
query_d = '''
           SELECT column_name, column_default, data_type 
           FROM INFORMATION_SCHEMA.COLUMNS 
           WHERE table_name = 'authors';
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,column_name,column_default,data_type
0,author_id,None,integer
1,author,None,text


В таблице authors собрана информация о 636 авторах и состоит из столбцов "идентификатор автора" и "имя автора". Дублирующихся значений нет, данные представлены в корректных форматах.

In [11]:
query = '''
           SELECT *
           FROM publishers
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [12]:
query = '''
           SELECT COUNT(*)
           FROM publishers
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,count
0,340


In [13]:
query_d = '''
           SELECT * 
           FROM publishers 
           WHERE ctid NOT IN (SELECT max(ctid) FROM publishers GROUP BY publisher_id)
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,publisher_id,publisher


In [14]:
query_d = '''
           SELECT column_name, column_default, data_type 
           FROM INFORMATION_SCHEMA.COLUMNS 
           WHERE table_name = 'publishers';
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,column_name,column_default,data_type
0,publisher_id,None,integer
1,publisher,None,text


Таблица publishers содержит данные об идентификаторах издательств и их названий, которых насчитывается 340. Дубликатов не имеется.

In [15]:
query = '''
           SELECT *
           FROM ratings
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [16]:
query = '''
           SELECT COUNT(*)
           FROM ratings
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,count
0,6456


In [17]:
query_d = '''
           SELECT * 
           FROM ratings 
           WHERE ctid NOT IN (SELECT max(ctid) FROM ratings GROUP BY rating_id)
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,rating_id,book_id,username,rating


In [18]:
query_d = '''
           SELECT column_name, column_default, data_type 
           FROM INFORMATION_SCHEMA.COLUMNS 
           WHERE table_name = 'ratings';
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,column_name,column_default,data_type
0,rating_id,None,integer
1,book_id,None,integer
2,username,None,text
3,rating,None,integer


В таблице ratings представлены данные о 6456 пользовательских оценках книг: идентификатор оценки, идентификатор книги, имя пользователя, оставившего оценку, и оценка книги.

In [19]:
query = '''
           SELECT *
           FROM reviews
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [20]:
query = '''
           SELECT COUNT(*)
           FROM reviews
        '''  
pd.io.sql.read_sql(query, con = engine).head()

,count
0,2793


In [21]:
query_d = '''
           SELECT * 
           FROM reviews 
           WHERE ctid NOT IN (SELECT max(ctid) FROM reviews GROUP BY review_id)
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,review_id,book_id,username,text


In [22]:
query_d = '''
           SELECT column_name, column_default, data_type 
           FROM INFORMATION_SCHEMA.COLUMNS 
           WHERE table_name = 'reviews';
        '''  
pd.io.sql.read_sql(query_d, con = engine).head()

,column_name,column_default,data_type
0,review_id,None,integer
1,book_id,None,integer
2,username,None,text
3,text,None,text


Таблица reviews содержит данные о пользовательских обзорах: идентификатор обзора, идентификатор книги, имя автора обзора, текст обзора. Таких обзоров насчитывается 2793.

Далее посчитаем сколько было выпущено книг после 1 января 2000 года.

In [23]:
query_1 = '''
           SELECT COUNT(title) as sum_title
           FROM books
           WHERE publication_date > '2000-01-01'
        '''  
query_1 = pd.io.sql.read_sql(query_1, con = engine) 
query_1

,sum_title
0,819


После 1 января 2000 года было выпущено 819 книг.

Далее посчитаем для каждой книги количество обзоров и среднюю оценку.

In [24]:
query_2 = '''
           SELECT b.title,
                  COUNT(rev.review_id) as count_review,
                  ROUND(AVG(rat.rating), 2) as avg_rating
           FROM books as b
           LEFT JOIN reviews as rev ON b.book_id=rev.book_id
           LEFT JOIN ratings as rat ON b.book_id=rat.book_id
           GROUP BY b.book_id
           ORDER BY count_review DESC, avg_rating
        '''  
query_2 = pd.io.sql.read_sql(query_2, con = engine)
query_2

,title,count_review,avg_rating
0,Twilight (Twilight #1),1120,3.66
1,The Hobbit or There and Back Again,528,4.13
2,The Catcher in the Rye,516,3.83
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.41
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.29
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,Leonardo's Notebooks,0,4.00
997,Essential Tales and Poems,0,4.00
998,Disney's Beauty and the Beast (A Little Golden...,0,4.00


Мы вывели книги с самым большим количеством отзывов с максимальной оценкой.

Следующим шагом определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [25]:
query_p = '''
           SELECT p.publisher,
                  COUNT(b.book_id) as count_books
           FROM publishers as p
           LEFT JOIN books as b ON p.publisher_id=b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY count_books DESC
        '''  
query_p = pd.io.sql.read_sql(query_p, con = engine)
query_p.head(1)

,publisher,count_books
0,Penguin Books,42


Издательство Penguin Books выпускает больше всего книг с толщиной более 50 страниц.

Далее определим автора с самой высокой средней оценкой книг. Учтем только книги с 50 и более оценками.

In [26]:
query_4 = '''
           SELECT aut.author,
                  ROUND(AVG(rat.rating), 2) as avg_rating
           FROM authors as aut
           JOIN books as b ON aut.author_id=b.author_id
           JOIN ratings as rat ON b.book_id=rat.book_id
           GROUP BY aut.author
           HAVING COUNT(rat.rating_id) > 50
           ORDER BY avg_rating DESC
        '''  
query_4 = pd.io.sql.read_sql(query_4, con = engine)
query_4.head(1)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


J.K. Rowling и художница Mary GrandPré публикуют книги с самой высокой пользовательской оценкой. По всей видимости речь идет о серии книг о Гарри Поттере.

Следующим шагом посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [27]:
query_rev = '''
           SELECT ROUND(AVG(cr.count_reviews), 2)
           FROM (SELECT username,
                        COUNT(review_id) as count_reviews
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating_id) > 50)
                 GROUP BY username) as cr
             
        '''  
query_rev = pd.io.sql.read_sql(query_rev, con = engine)
query_rev.head(50)

,round
0,24.33


В среднем пользователи, которые ставят больше 50 оценок, пишут 24,33 обзора на прочитанную литературу.